In [ ]:
import mqr
from mqr.plot import Figure

In [ ]:
from IPython.display import display, HTML

# Datasets

In [ ]:
import pandas as pd

# Random glue data
data = pd.read_csv(mqr.sample_data('anova-glue.csv'), index_col='Run')

---
# ANOVA

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
# Simple one factor
model = ols('adhesion_force ~ C(primer) + C(glue) + C(primer):C(glue)', data=data)
result = model.fit()
# display(result.summary())
table = sm.stats.anova_lm(result, typ=2)
table.iloc[:, 3] *= 100 # Pct
table.rename(columns={'PR(>F)': 'P(>F) %'}, inplace=True)
table.loc['Total', 'sum_sq':'df'] = table.sum(axis=0) # Add a total row
table

In [ ]:
result.summary().tables[0]

In [ ]:
display(mqr.anova.groups(data, value='adhesion_force', factor='primer'))
display(mqr.anova.groups(data, value='adhesion_force', factor='glue'))

---
## Residual analysis

In [ ]:
# See https://en.wikipedia.org/wiki/Studentized_residual#Internal_and_external_studentization

influence = result.get_influence()
N = influence.nobs
with Figure(8, 4) as (fig, ax):
    ax.plot(influence.resid_studentized_internal, linewidth=0, marker='o', fillstyle='none')
    ax.plot(influence.resid_studentized_external, linewidth=0, marker='x')
    ax.set_ylabel('residuals')

    axt = ax.twinx()
    axt.bar(range(N), 1-influence.cooks_distance[1], alpha=0.5)
    axt.set_ylim(0.0, 1.0)
    axt.set_ylabel("Cook's distance (1-p)")

    ax.legend(['stud. internal', 'stud. external'])
    ax.grid(axis='y')

In [ ]:
with Figure(10, 6, 2, 2, height_ratios=[2, 1]) as (fig, ax):
    mqr.plot.regression.residuals(result, tr='studentised_external', influence_stat='cooks_dist', axs=ax)